Initiate esipy objects with dev configs and pyswagger cache then demonstrate operability by checking the status of the ccp api server

In [1]:
from config import *
from esipy import EsiApp, EsiClient, EsiSecurity, cache
import pandas as pd
import json, itertools
from pprint import PrettyPrinter

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.expand_frame_repr', False)

f_cache = cache.FileCache(path="./f_cache")
# create the eve app interface
esiapp = EsiApp(cache=f_cache).get_latest_swagger

# init the security object
esisecurity = EsiSecurity(redirect_uri=ESI_CALLBACK,
                          client_id=ESI_CLIENT_ID,
                          secret_key=ESI_SECRET_KEY,
                          headers={'User-Agent': ESI_USER_AGENT})

# init the client
esiclient = EsiClient(security=esisecurity,
                      cache=f_cache,
                      headers={'User-Agent': ESI_USER_AGENT})

#  server_request_names = [k for k in esiapp.op.keys()] OR https://esi.evetech.net/
ccp_resp = esiclient.request(esiapp.op['get_status']())
print(f'Tranquility Status\nresp_code: {ccp_resp.status}\n{ccp_resp.data}')

def get_unique_region_ids():
    return pd.read_parquet("everefread")["regionID"].unique().tolist()


def req_all_region_order_pages():
    ops = []
    for id in get_unique_region_ids():
        ops.append(esiapp.op['get_markets_region_id_orders'](region_id=id))
    results = []
    for rq, rs in esiclient.multi_request(ops, raw_body_only=True):
        if rs.status == 200:
            pages = rs.header['X-Pages'][0]
            region = rq._Request__p.get('path')['region_id']
        results.append((region, pages))
    return results


def gen_region_market_requests(reg_id_pages:tuple):
    reg_id = reg_id_pages[0]
    pages = reg_id_pages[1]
    esiapp.op['get_markets_region_id_orders'](region_id=reg_id)
    operations = []
    for page in range(1, pages + 1):
        operations.append(esiapp.op['get_markets_region_id_orders'](
                region_id=reg_id, page=page))
    return operations

c:\Users\Mothership\Desktop\HermitKrabacus\hkvenv\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Tranquility Status
resp_code: 200
{'players': 16228, 'server_version': '2176644', 'start_time': <pyswagger.primitives._time.Datetime object at 0x000001C1B9190370>}


Next we request all region market ids and page counts.<br>
build a flat list of requests to send concurrently via esiclient<br>
send it and save resp in memory

In [2]:
ids_and_pages = req_all_region_order_pages()
req_bucket = list(map(gen_region_market_requests,ids_and_pages))
flat_req_bucket = list(itertools.chain(*req_bucket))
market_responses = esiclient.multi_request(flat_req_bucket, raw_body_only=True)

[https://esi.evetech.net/latest/markets/10000032/orders/?datasource=tranquility&order_type=all&page=4] returned expired result: {'Date': 'Fri, 09 Dec 2022 01:57:22 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'Content-Type,Authorization,If-None-Match,X-User-Agent', 'Access-Control-Allow-Methods': 'GET,HEAD,OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Type,Warning,ETag,X-Pages,X-ESI-Error-Limit-Remain,X-ESI-Error-Limit-Reset', 'Access-Control-Max-Age': '600', 'Allow': 'GET,HEAD,OPTIONS', 'Cache-Control': 'public', 'Content-Encoding': 'gzip', 'Etag': '"cb81e9a2e23597c4bd67bc52b106e038670d7438eed53d5ceae84ff5"', 'Expires': 'Fri, 09 Dec 2022 01:57:22 GMT', 'Last-Modified': 'Fri, 09 Dec 2022 01:52:22 GMT', 'Strict-Transport-Security': 'max-age=31536000', 'Vary': 'Accept-Encoding', 'X-Esi-Error-Limit-Rem

iterate over the json responses and build a dataframe with the unpacked orders and recovered region_id that goes with it

In [3]:
total_eve_orders = []
for rq, rs in market_responses:
    if rs.status == 200:
        for rec in json.loads(rs.raw):
            region = rq._Request__p.get('path')['region_id']
            total_eve_orders.append({'region':region,**rec})
pp = PrettyPrinter(width=180, compact=True)
pp.pprint(pd.DataFrame(total_eve_orders))

           region  duration  is_buy_order                issued    location_id  min_volume    order_id       price   range  system_id  type_id  volume_remain  volume_total
0        10000001        90         False  2022-11-30T07:46:08Z       60012124           1  6384844802        9.99  region   30000005      219          32000         50000
1        10000001        90         False  2022-10-20T17:48:08Z       60014452           1  6335605419   900000.00  region   30000112     2869            100           100
2        10000001        90         False  2022-11-08T16:39:37Z       60012145           1  6370426882        5.50  region   30000082      266           9000         20000
3        10000001        90         False  2022-11-10T05:16:37Z       60014452           1  6371551917   982500.00  region   30000112     2531             10            10
4        10000001        90         False  2022-11-16T03:54:22Z       60012163           1  6376423446  9000000.00  region   30000044    112

analyze the raw data in the dataframe and build our view from scratch.<br>
we will use the following columns:<br>
* region
* type_id
* price
* volume_remain
* location_id
* system_id
* is_buy_order

we will use the following derived columns:<br>
*sell_order_value = price * volume_remain*<br>

omit all buy orders from analysis<br>
iterate the view over all unique regions<br>
    get regional order count from view shape[0]<br>
    get most popular location_id from view as regional_hub with location_id.mode()<br>
    get most popular system_id from view as solar_system hub with system_id.mode()<br>
    get hub_market_value from location_id view as sum of sell_order_value<br>
    get hub_orders_count from location_id view with shape[0]<br>
    get hub_unique_items from location_id view as type_id.unique<br>

build a list(dict()) from the regional hub analysis<br>

In [4]:
total_eve_orders = pd.DataFrame(total_eve_orders)
total_eve_orders['sell_order_value'] = total_eve_orders.price * total_eve_orders.volume_remain
total_eve_sell_orders = total_eve_orders[total_eve_orders.is_buy_order == False]

systems = []

for reg_id in total_eve_sell_orders.region.unique():
    region_sell_orders = total_eve_sell_orders[total_eve_sell_orders.region == reg_id]
    regional_orders_count = region_sell_orders.shape[0]
    regional_hub_id = region_sell_orders.location_id.mode()[0]
    solar_system = region_sell_orders[region_sell_orders.location_id ==
                                        regional_hub_id].system_id.mode()[0]
    hub_sell_orders = region_sell_orders[region_sell_orders.location_id ==
                                        regional_hub_id]

    hub_market_value = hub_sell_orders.sell_order_value.sum()
    hub_orders_count = hub_sell_orders.shape[0]
    hub_unique_items = hub_sell_orders.type_id.unique().shape[0]


    systems.append({'region': int(reg_id), 'regional_hub': int(regional_hub_id),'unique_items': int(hub_unique_items), 'regional_order_count': int(regional_orders_count), 'hub_total_orders': int(hub_orders_count), 'hub_market_value': float(hub_market_value)})


use list(dict()) to build a dataframe from the regional hub analysis<br>
calculate the regional hub market orders share as a percentage of the total market orders<br>
get names to region_id, location_id from the eve api<br>
merge all responses to final dataframe<br>

In [5]:
pp = PrettyPrinter(width=150,compact=True)
df_orders = pd.DataFrame(systems)
df_orders['hub_regional_order_ratio'] = round((df_orders.hub_total_orders / df_orders.regional_order_count) * 100,1)
id_list = df_orders.regional_hub.to_list()
req = esiapp.op['post_universe_names'](ids=id_list)
ccp_resp = esiclient.request(req)
with_sta_names = pd.DataFrame().from_records(ccp_resp.data).merge(
    df_orders, left_on='id', right_on='regional_hub').drop(columns=['id', 'category'])

id_list = with_sta_names.region.to_list()
req = esiapp.op['post_universe_names'](ids=id_list)
ccp_resp = esiclient.request(req)
pp.pprint(pd.DataFrame().from_records(ccp_resp.data).merge( with_sta_names, left_on='id', right_on='region').sort_values('hub_total_orders', ascending=False).drop(columns=['id', 'category']).head(10))


          name_x                                             name_y    region  regional_hub  unique_items  regional_order_count  hub_total_orders    hub_market_value  hub_regional_order_ratio
27     The Forge     Jita IV - Moon 4 - Caldari Navy Assembly Plant  10000002      60003760         14231                213134            181558  175823683775410.22                     85.20
31        Domain         Amarr VIII (Oris) - Emperor Family Academy  10000043      60008494          9654                104234             59371    9178950894632.57                     57.00
5    Sinq Laison  Dodixie IX - Moon 20 - Federation Navy Assembl...  10000032      60011866          8367                 71753             35665  135780429637891.58                     49.70
4       Heimatar           Rens VI - Moon 8 - Brutor Tribe Treasury  10000030      60004588          6511                 51125             20313    2656145236958.77                     39.70
30    Metropolis    Hek VIII - Moon 12 -

In [79]:
# get the top 3 locations in each region by sell order value
df = total_eve_sell_orders.groupby(['region', 'location_id']).agg({
        'order_id': 'count',
        'sell_order_value': 'sum'
    }).reset_index().sort_values('order_id', ascending=False).groupby('region').head(3)
df['order_sum'] = df.groupby('region')['order_id'].transform(sum)
df.sort_values(by=['order_sum', 'order_id'], ascending=[False, False]).drop('order_sum', axis=1).head(30)


,region,location_id,order_id,sell_order_value
314,10000002,60003760,181558,175823683775410.22
427,10000002,60015027,1741,18965107227113.41
408,10000002,60012667,652,28192404426.97
3420,10000043,60008494,59371,9178950894632.57
3294,10000043,60006658,2213,4729460130346.45
3527,10000043,60015021,1259,16964599980698.64
1837,10000032,60011866,35665,135780429637891.59
1885,10000032,60015016,1540,19539883214139.28
1878,10000032,60014740,933,18131401351685.75
1440,10000030,60004588,20313,2656145236958.77
